## Homework
### Noe: Data for tasks 1 and 2 could be found [here](https://syncandshare.lrz.de/getlink/fiLmDyv8FXqFyN1X3hbhwazH/01-Realsense)

### 2. Object Twin (workload 3 students):
In this exercise, we will load a realsense-viewer rosbag recording, then use opencv and pyrender to create a twin of a moving checkerboard.

     


In [1]:
#####################################################
##               Read bag from file                ##
#####################################################
# First import library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
# Import argparse for command-line options
import argparse
# Import os.path for file path manipulation
import os.path
import copy as cp

1. Loading color and depth data:
     * Use pyrealsense2 to read the bagfile and acquire color, depth, aligned depth to color, color camera intrinsics, depth camera intrinsics. (Show the images in a loop using `cv2.imshow`)

In [4]:
# Read from Stereo Module
bag_input = '../Homework/HW1-2-data/20220405_220626.bag'

# Create pipeline
pipeline = rs.pipeline()

# Create a config object
config = rs.config()

# Tell config that we will use a recorded device from file to be used by the pipeline through playback.
rs.config.enable_device_from_file(config, bag_input)

# Configure the pipeline to stream the depth stream
# Change this parameters according to the recorded bag file resolution
config.enable_stream(rs.stream.depth, 848, 480,rs.format.z16, 30)
config.enable_stream(rs.stream.color,rs.format.rgb8, 30)

# Start streaming from file
pipeline.start(config)

In [5]:
# Create colorizer object
colorizer = rs.colorizer()

profile = pipeline.get_active_profile()
print(profile)
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
print("depth_profile: ",depth_profile)
color_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))
print("color_profile: ", color_profile)
depth_intrinsics = depth_profile.get_intrinsics()
color_intrinsics = color_profile.get_intrinsics()
print('depth_intrinsics:', depth_intrinsics)
print('color_intrinsics:', color_intrinsics)
w, h = depth_intrinsics.width, depth_intrinsics.height
print('Width and Height:',w,h)


depth_profile:  <pyrealsense2.video_stream_profile: Depth(0) 848x480 @ 30fps Z16>
color_profile:  <pyrealsense2.video_stream_profile: Color(0) 640x480 @ 30fps RGB8>
depth_intrinsics: [ 848x480  p[426.785 234.17]  f[419.127 419.127]  Brown Conrady [0 0 0 0 0] ]
color_intrinsics: [ 640x480  p[323.426 249.374]  f[607.323 606.302]  Inverse Brown Conrady [0 0 0 0 0] ]
Width and Height: 848 480


In [6]:
# Define detectors parameters
pattern_size = (9, 6)
board_pattern = [9, 6]
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
imgpoints = [] # 2d points in image plane.

# Get camera matrix and distortion coefficients
camera_matrix = np.array([[color_intrinsics.fx, 0, color_intrinsics.ppx], [0, color_intrinsics.fy, color_intrinsics.ppy], [0, 0, 1]])
dist_coeffs = np.array([0., 0., 0., 0., 0.])

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((board_pattern[1] * board_pattern[0],3), np.float32)
objp[:,:2] = np.mgrid[0:board_pattern[0],0:board_pattern[1]].T.reshape(-1,2)

2. Checkerboard detection and tracking: 
     * The checkerboard has a `6x9` pattern where each square has an edge length of 4 cm.
     * Using opencv we want Find its corners (use `cv2.findChessboardCorners`, and `cv2.cornersSubPix`). then use `cv2.drawChessboardCorners` to overlay the detections on the colored image
     * From the previous step, you will have 2D/3D correspondences for the corners. Use `cv2.solvePnP` to estimate the object to camera translation and rotation vectors.
     * *Extra:* Use opencv drawing utils and perspective projection function to draw a 3D axis, and a cropping mask for the board. Useful functions here could be `cv2.line,cv2.projectPoints,cv2.fillPoly`.
3. Modeling the checkerboard in pyrender:
    * Using pyrender create a scene with camera and a `Box` mesh corresponding to the checkerboard.
    * Notes:
      1. You will need to scale the box and shift its center to match the checkerboard 3d coordinate system in opencv
      2. To convert from opencv camera to pyrender camera in you system you may need to rotate your objects by 90 degees around the X-axis (depending on your implementation) 
4. Visualization:
    * In the loop, update the mesh pose with the updated pose of the checkerboard
    * Compare the rendered depth value to the actual algined_depth values we got from realsense.

In [7]:
def draw(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
# axis = np.float32([[8,0,0], [0,5,0], [0,0,-3]]).reshape(-1,3)
axis = np.float32([[0,0,0], [0,5,0], [8,5,0], [8,0,0],
                   [0,0,-3],[0,5,-3],[8,5,-3],[8,0,-3] ])
# Streaming loop
while True:
    # Get frameset of depth
    frames = pipeline.wait_for_frames()

    # Get depth frame
    depth_frame = frames.get_depth_frame()
    
    # Get color frame
    color_frame = frames.get_color_frame()

    
    # Colorize depth frame to jet colormap
    depth_color_frame = colorizer.colorize(depth_frame)

    # Convert depth_frame to numpy array to render image in opencv
    depth_color_image = np.asanyarray(depth_color_frame.get_data())
    
    # get color frame
    color_image = np.asanyarray(color_frame.get_data())
    color_image2 = cp.deepcopy(color_image)
    # get gray frame
    gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    
    # Detect Chessboard Corners
    ret, corners = cv2.findChessboardCorners(gray_image, board_pattern, None)
    
    if ret == True:
        # Refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray_image, corners, (11,11),(-1,-1), criteria)
        objpoints.append(objp)
        imgpoints.append(corners)
        
        # SolvePnP and project back2 2D
        success, rotation_vector, translation_vector = cv2.solvePnP(objp, corners, camera_matrix, dist_coeffs, flags=0)
        imgpoints2, _ = cv2.projectPoints(axis, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
        
        # 2.1 Draw and display the corners
        cv2.drawChessboardCorners(color_image, board_pattern, corners2, ret)
        cv2.imshow('Corners', color_image)
        
        # 2.2 Draw 3D axis
        corners2 = corners2.astype(int)
        imgpoints2 = imgpoints2.astype(int)
        color_image2 = draw(color_image2,corners2,imgpoints2) 
#         cv2.fillPoly(color_image2, pts = [contours], color =(0,255,0))
        cv2.imshow("filledPolygon", color_image2)
        

    key = cv2.waitKey(1)
    # if pressed escape exit program
    if key == 27:
        cv2.destroyAllWindows()
        break
np.save('objpoints.npy', objpoints)
np.save('imgpoints.npy', imgpoints)

## References and Resources
[1]. https://www.intelrealsense.com/stereo-depth-vision-basics/

[2]. https://dev.intelrealsense.com/docs/intel-realsensetm-d400-series-calibration-tools-user-guide

[3]. https://dev.intelrealsense.com/docs/whitepapers

[4]. https://docs.opencv.org/4.x/

[5]. https://pyrender.readthedocs.io/en/latest/examples/quickstart.html

[6]. https://wiki.ros.org/noetic

[7]. https://calib.io/blogs/knowledge-base/camera-models

[8]. https://web.stanford.edu/class/cs231a/course_notes/03-epipolar-geometry.pdf

[9]. https://dev.intelrealsense.com/docs/intel-realsensetm-d400-series-calibration-tools-user-guide